# 1. Google drive 연동


In [ ]:
import os
from google.colab import drive
# google mount
drive.mount('/content/gdrive')
os.chdir('./gdrive/My Drive/chatbot/KoGPT2')

Mounted at /content/gdrive


# 2. GPU 확인

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
! nvidia-smi

Wed Dec  9 23:10:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 2. 1 필요 패키지 설치

In [ ]:
os.getcwd()

'/content/gdrive/My Drive/chatbot/KoGPT2'

In [ ]:
! git clone https://github.com/SKT-AI/KoGPT2.git
cd KoGPT2
! pip install -r requirements.txt
! pip install .

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import torch
import torch.nn as nn
import os
import torchtext
import time
import datetime
from tqdm import tqdm
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer

     |████████████████████████████████| 256kB 5.4MB/s 
     |████████████████████████████████| 68.7MB 94kB/s 
     |████████████████████████████████| 1.1MB 44.6MB/s 
     |████████████████████████████████| 752.0MB 23kB/s 
     |████████████████████████████████| 675kB 52.7MB/s 
     |████████████████████████████████| 890kB 47.4MB/s 
     |████████████████████████████████| 3.8MB 48.6MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470040 sha256=8e601d7a840a58faa598671b3b592bc81d002fe4c726c46cde9d461ad3f26ad2
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=feb76483190cffe7932c14193282f8d658e1d6febb6fee4e203ba8ba22cbf2be
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built gluonnlp sacremoses
ERROR: torchvision 0.8.1+cu101

# 3. Data 불러오기
1. Chatbot_data	일상 챗봇 학습용 문답 페어 11,876건
일상(0) / 이별,부정(1) / 사랑,긍정(2) 라벨 부여 

# 3.1 데이터 전처리

In [ ]:
os.chdir('..')
tok_path = get_tokenizer()
model, vocab = get_pytorch_kogpt2_model()
tokenizer = SentencepieceTokenizer(tok_path,  num_best=0, alpha=0)

In [ ]:
tok_path = get_tokenizer()
# model, vocab
model, vocab = get_pytorch_kogpt2_model()
# tokenizer
tokenizer = SentencepieceTokenizer(tok_path,  num_best=0, alpha=0)
# bos token, eos token 붙이기
class make_data_set:
    def __init__(self):
        '''
        data : data frame
        data - Q, A 로 구성 
        '''
        data=pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv',sep=',',header=0)
        # <usr> sentence <cls>
        data['Q_input_id']=data['Q'].apply(lambda i :[vocab['<usr>']]+vocab[tokenizer(i)]+[vocab['<unused0>']])
        # <sys> sentence </s>
        data['A_input_id']=data['A'].apply(lambda i :[vocab['<sys>']]+vocab[tokenizer(i)]+[vocab['</s>']])
        data['cls_position']=None
        Max = config.max_len
        for i in data.index:
            # Q+A의 길이가 Max보다 길면 Q은 -Max//2부터 끝까지.
            if len(data.Q_input_id[i])+len(data.A_input_id[i])>Max:
                data.Q_input_id[i]=data.Q_input_id[i][-(Max//2):]
                data.A_input_id[i]=data.A_input_id[i][:(Max-(Max//2))]
            # data['attention_mask'][i] = [1] * (len(data.Q_input_id[i])+len(data.A_input_id[i]))+[0] * (Max - (len(data.Q_input_id[i])+len(data.A_input_id[i])))
            # data['token_id'][i] = [0] * len(data.Q_input_id[i]) + [1] * (Max-len(data.Q_input_id[i]))
            # classification을 위해서
            data['cls_position'][i] = len(data.Q_input_id[i])-1 
        data['input_id']=data['Q_input_id']+data['A_input_id']
        # pad 하기
        data['input_id']=data['input_id'].apply(lambda j : j+(Max-len(j))*[vocab['<pad>']])
        # <usr> sentence <cls> | <sys> sentence  | <eos> pad ...
        # pad, pad , ... pad,  | sentence, <eos> | pad, pad ...

        data['lm_label']=data['Q_input_id'].apply(lambda i : len(i) * [vocab['<pad>']])+data['A_input_id'].apply(lambda i : i[1:])
        data['lm_label']=data['lm_label'].apply(lambda j : j+(Max-len(j))*[vocab['<pad>']])
        self.data=data

    def return_data_set(self):
        result=self.data.loc[:,['input_id','lm_label','cls_position','label']]#'attention_mask','token_id','cls_position','lm_label','label']]
        return result

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
using cached model


# 3.2 model load

In [ ]:
class Config(dict):
    __setattr__=dict.__setitem__
    __getattr__=dict.__getitem__
config=Config({'cls':3,'unk_idx':vocab[vocab.unknown_token],'bos_idx':vocab[vocab.bos_token],'eos_idx':vocab[vocab.eos_token],'cls_idx':vocab['<unused0>'],'usr_idx':vocab['<usr>'],'sys_idx':vocab['<sys>'],'padding_idx':vocab[vocab.padding_token],'n_vocab':len(vocab), 'batch_size':2, 'max_len':64})

In [ ]:
config=Config({'cls':3,'unk_idx':vocab[vocab.unknown_token],'bos_idx':vocab[vocab.bos_token],'eos_idx':vocab[vocab.eos_token],'cls_idx':vocab['<unused0>'],'usr_idx':vocab['<usr>'],'sys_idx':vocab['<sys>'],'padding_idx':vocab[vocab.padding_token],'n_vocab':len(vocab), 'batch_size':2, 'max_len':64})

In [ ]:
make_data=make_data_set(config)
dataset=make_data.return_data_set()
print(dataset.head())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                                            input_id  ... label
0  [2, 385, 47460, 47437, 49108, 47812, 6, 4, 332...  ...     0
1  [2, 106, 47445, 47766, 1084, 1024, 47816, 4748...  ...     0
2  [2, 141, 47650, 47514, 47471, 2211, 47593, 299...  ...     0
3  [2, 141, 47650, 47514, 47471, 1057, 2211, 4759...  ...     0
4  [2, 753, 9003, 415, 20572, 6, 4, 637, 10855, 7...  ...     0

[5 rows x 4 columns]


### 데이터 저장

In [ ]:
dataset.to_csv('./chatbot_data.csv')

### train test split
test는 0.2 비율

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(dataset,test_size=0.2)
val,test=train_test_split(test, test_size = 0.4)
# 얘네도 저장
train.to_pickle('./train_chatbot')
test.to_pickle('./test_chatbot')
val.to_pickle('./val_chatbot')

In [ ]:
# 불러오기
train = pd.read_pickle('./train_chatbot')
val = pd.read_pickle('./val_chatbot')
test = pd.read_pickle('./test_chatbot')

### DataLoader

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
train_dataset=TensorDataset(torch.LongTensor(train.input_id.values.tolist()),torch.LongTensor(train.cls_position.values.tolist()),torch.LongTensor(train.label.values.tolist()),torch.LongTensor(train.lm_label.values.tolist()))
val_dataset=TensorDataset(torch.LongTensor(val.input_id.values.tolist()),torch.LongTensor(val.cls_position.values.tolist()),torch.LongTensor(val.label.values.tolist()),torch.LongTensor(val.lm_label.values.tolist()))
test_dataset=TensorDataset(torch.LongTensor(test.input_id.values.tolist()),torch.LongTensor(test.cls_position.values.tolist()),torch.LongTensor(test.label.values.tolist()),torch.LongTensor(test.lm_label.values.tolist()))
# train loader, test loader
train_loader=DataLoader(train_dataset,batch_size=config.batch_size,drop_last=True)
val_loader=DataLoader(val_dataset,batch_size=config.batch_size,drop_last=True)
test_loader=DataLoader(test_dataset,batch_size=config.batch_size,drop_last=True)

# 3.3 Model

In [ ]:
class my_kogpt2_model(nn.Module):
    def __init__(self, config, model):
        super().__init__()
        self.config = config
        self.model = model
        self.classification = nn.Linear(self.config.n_vocab, self.config.cls)

    def inference(self,
               input_ids,
               ):
        pred=self.model.generate(input_ids=input_ids,eos_token_id=vocab['</s>'],bos_token_id = vocab['<usr>'], pad_token_id = config.padding_idx, decoder_start_token_id=vocab['<sys>'],do_sample=True, max_length=64, top_p=0.92, top_k=50, temperature=0.6, no_repeat_ngram_size=None, num_return_sequences=1, early_stopping=False)
        # input ids : 1, seq_len
        with torch.no_grad():
            model.eval()
            cls_position = input_ids.squeeze(0).tolist().index(config.cls_idx)
            output = self.model.forward(input_ids=input_ids)[0]
            cls_output = self.classification.forward(output[:,cls_position,:])
        return pred.squeeze().tolist(),cls_output.argmax(-1).item()

    def forward(self,data):
        input_ids = data[0]
        cls_position = data[1]
        #lm_label = data[-1]
        # batch size, seq len, n_vocab
        output = self.model.forward(input_ids=input_ids)[0]
        e=torch.cat([output[_,j,:].unsqueeze(0) for _,j in enumerate(cls_position.tolist())],0)
        cls_output = self.classification.forward(e)
        return output, cls_output

# 4. 학습

In [ ]:
import time,datetime
def Eplased(dt):
    d=int(round(dt))
    return str(datetime.timedelta(seconds=d))

In [ ]:
device='cuda'
print(device)
epochs = 100         # Num of Epoch
learning_rate = 1e-5
Model = my_kogpt2_model(config, model)
Model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate) 
criterion1 = nn.CrossEntropyLoss(ignore_index=config.padding_idx,reduction = 'sum') # language modeling
criterion2 = nn.CrossEntropyLoss() # classification

cuda


In [ ]:
start_time = time.time()
min_value = None
for epoch in tqdm(range(1,epochs+1),desc='epoch',mininterval=600):
    print('\n')
    print(epoch)
    with torch.no_grad():
        val_total_loss=0.
        val_loss_lm = 0.
        val_loss_cls = 0.
        for data in val_loader:
            model.eval()
            data = tuple(i.to(device) for i in data)
            lm_output, cls_output = Model.forward(data)        
            loss1 = criterion1(lm_output.transpose(1,2),data[-1])
            loss2 = criterion2(cls_output,data[-2])
            loss = (loss1/config.batch_size+loss2)
            val_loss_lm += (loss1/config.batch_size).item()
            val_loss_cls += loss2.item()
            val_total_loss+=loss.item()
            
        if min_value is None:
            min_value = val_total_loss/len(val_loader) 
        if min_value < val_total_loss/len(val_loader):
            print('early_stop')
            print(epoch)
            torch.save(Model.state_dict(),'./chatbot1_epoch_%d'%epoch)
            break
        else:
            min_value = val_total_loss/len(val_loader)
    total_loss=0.
    loss_lm = 0.
    loss_cls = 0.
    for data in tqdm(train_loader,desc='step',mininterval=60):
        model.train()
        optimizer.zero_grad()
        data = tuple(i.to(device) for i in data)
        lm_output, cls_output = Model.forward(data)        
        loss1 = criterion1(lm_output.transpose(1,2),data[-1])
        loss2 = criterion2(cls_output,data[-2])
        loss = (loss1/config.batch_size+loss2)
        loss.backward()
        optimizer.step()
        loss_lm += (loss1/config.batch_size).item()
        loss_cls += loss2.item()
        total_loss += loss.item()
    if epoch%5==0:
        print('epoch : %d'%epoch)
        print('total loss : %.3f'%(total_loss/len(train_loader)))
        print('lm loss : %.3f'%(loss_lm/len(train_loader)))
        print('cls loss : %.3f'%(loss_cls/len(train_loader)))
        
        print('val total loss  : %.3f'%(val_total_loss/len(val_loader)))
        print('val lm loss : %.3f'%(val_loss_lm/len(val_loader)))
        print('val cls loss : %.3f'%(val_loss_cls/len(val_loader)))
        print('Eplased Time : %s'%(Eplased(time.time()-start_time)))

In [ ]:
import smtplib
from email.mime.text import MIMEText
s = smtplib.SMTP('smtp.gmail.com',587)
s.starttls()
s.login('tillcry8@gmail.com','xbcohxszwogqcrfj')
msg = MIMEText('complete')
msg['Subject'] = 'chatbot'
s.sendmail("tillcry8@gmail.com", "tillcry8@gmail.com", msg.as_string())
s.quit()

(221, b'2.0.0 closing connection u9sm3476235edd.54 - gsmtp')

# Test data로 추정해보자

## Model load

In [ ]:
! ls

chatbot1_epoch_13  T0_data	T5_tweeter.csv	train	       val
chatbot_data.csv   T5_tweeter	test		train_chatbot  val_chatbot
KoGPT2		   T5_tweeter_	test_chatbot	tweeter.csv


In [ ]:
device='cuda'
Model = my_kogpt2_model(config, model)
Model.to(device)
criterion1 = nn.CrossEntropyLoss(ignore_index=config.padding_idx,reduction = 'sum') # language modeling
criterion2 = nn.CrossEntropyLoss() # classification
Model.load_state_dict(torch.load('./chatbot1_epoch_13'))

<All keys matched successfully>

In [ ]:
with torch.no_grad():
    total_loss=0.
    loss_lm = 0.
    loss_cls = 0.
    total_acc = 0.
    for data in tqdm(train_loader,desc='step',mininterval=60):
        model.eval()
        data = tuple(i.to(device) for i in data)
        lm_output, cls_output = Model.forward(data)        
        loss1 = criterion1(lm_output.transpose(1,2),data[-1])
        loss2 = criterion2(cls_output,data[-2])
        loss = (loss1/config.batch_size+loss2)
        loss_lm += (loss1/config.batch_size).item()
        loss_cls += loss2.item()
        total_loss += loss.item()
        total_acc += (cls_output.argmax(-1)==data[-2]).float().mean()
    print('total_loss')
    print(total_loss/len(train_loader))
    print('total_lm')
    print(loss_lm/len(train_loader))
    print('total_cls')
    print(loss_cls/len(train_loader))
    print('total_acc')
    print(total_acc/len(train_loader))


step: 100%|██████████| 4729/4729 [01:28<00:00, 53.15it/s]

total_loss
6.574096047325577
total_lm
6.43995144562455
total_cls
0.13414459955163666
total_acc
tensor(0.9519, device='cuda:0')


In [ ]:
with torch.no_grad():
    total_loss=0.
    loss_lm = 0.
    loss_cls = 0.
    total_acc = 0.
    for data in tqdm(val_loader,desc='step',mininterval=60):
        model.eval()
        data = tuple(i.to(device) for i in data)
        lm_output, cls_output = Model.forward(data)        
        loss1 = criterion1(lm_output.transpose(1,2),data[-1])
        loss2 = criterion2(cls_output,data[-2])
        loss = (loss1/config.batch_size+loss2)
        loss_lm += (loss1/config.batch_size).item()
        loss_cls += loss2.item()
        total_loss += loss.item()
        total_acc += (cls_output.argmax(-1)==data[-2]).float().mean()
    print('total_loss')
    print(total_loss/len(val_loader))
    print('total_lm')
    print(loss_lm/len(val_loader))
    print('total_cls')
    print(loss_cls/len(val_loader))
    print('total_acc')
    print(total_acc/len(val_loader))


step: 100%|██████████| 709/709 [00:13<00:00, 52.67it/s]

total_loss
19.764458034371454
total_lm
19.329035978895654
total_cls
0.4354220768067184
total_acc
tensor(0.8533, device='cuda:0')


In [ ]:
with torch.no_grad():
    total_loss=0.
    loss_lm = 0.
    loss_cls = 0.
    total_acc = 0.
    for data in tqdm(test_loader,desc='step',mininterval=60):
        model.eval()
        data = tuple(i.to(device) for i in data)
        lm_output, cls_output = Model.forward(data)        
        loss1 = criterion1(lm_output.transpose(1,2),data[-1])
        loss2 = criterion2(cls_output,data[-2])
        loss = (loss1/config.batch_size+loss2)
        loss_lm += (loss1/config.batch_size).item()
        loss_cls += loss2.item()
        total_loss += loss.item()
        total_acc += (cls_output.argmax(-1)==data[-2]).float().mean()
    print('total_loss')
    print(total_loss/len(test_loader))
    print('total_lm')
    print(loss_lm/len(test_loader))
    print('total_cls')
    print(loss_cls/len(test_loader))
    print('total_acc')
    print(total_acc/len(test_loader))


step: 100%|██████████| 473/473 [00:08<00:00, 54.17it/s]

total_loss
19.19323715372267
total_lm
18.747501539881586
total_cls
0.44573566905824846
total_acc
tensor(0.8573, device='cuda:0')


## 문장 생성과 함께 감정 상태 예측하기

### label
label이 0이면 일상, 1이면 부정/이별, 2이면 긍정/사랑
허나 그렇게 정확하지만은 않다.

In [ ]:
sentence_list = []
label_list = []
for j in test.index:
    sentence=vocab.to_tokens(test.input_id[j])
    label=test.label[j]
    label_list.append(label)
    sent = ''
    for i in sentence:
        if i == '<unused0>':
            break
        sent += i.replace('▁', ' ')
    sentence_list.append(sent.replace('<usr> ',''))
print(sentence_list[:10])
print(label_list[:10])

['짝남과 자연스럽게 친해질 수 있는 방법 있을까?', '말을 좀 잘 하고 싶어', '행복해져야해', '하고 싶은 게 없어', '시간이 지날수록 더 보고싶어', '학교 적을하고 잘 다닐 수 있을까?', '단체생활 힘들어', '쓸쓸하다', '이제 바람 분다', '오토바이 사고 싶어']
[2, 0, 1, 0, 1, 0, 0, 0, 0, 0]


In [ ]:
import random
idx = list(range(len(sentence_list)))
input = random.choice(idx)
print(input)
print(sentence_list[input])
print(label_list[input])

input_sentence = sentence_list[input]
input_label = label_list[input]

500
잊지 못하겟어
1


In [ ]:
input_sentence
input_label

1

In [ ]:
toked = tokenizer(input_sentence)
input_ids = torch.LongTensor([vocab['<usr>']]+vocab[toked]+[vocab['<unused0>']])
print(input_ids)

tensor([    2, 14037,   409, 47451, 49014, 47487,     6])


In [ ]:
pred, lm_output = Model.inference(input_ids.to(device))
print(pred)
print(lm_output)
result=''
start = pred.index(config.cls_idx)
for i in pred[start+1:]:
    k = vocab.to_tokens(i)
    if k == '</s>':
        break
    result+=k.replace('▁',' ')
print(result)
print(lm_output)

[2, 14037, 409, 47451, 49014, 47487, 6, 1256, 11196, 13212, 1712, 6214, 47444, 114, 7122, 47440, 1]
1
 그런 나를 보면서 어떻게 잊을 수 있을까.
1
